#  생성형 AI평가: 나라명을 입력하면 그 나라의 제일 유명한 음식의 레시피를 출력하는 복합체인 구현¶
위 : 나라설명 -> 나라이음 -> 수도(체인 두개 연결)
나라이름- 그 나라에서 제일 유명한 음식-> 레시피

In [1]:
# 허남식 생성형 AI 과제 - 음식 추천 및 레시피 생성 렝체인
# 평가문항: 생성형 AI를 이용한 체인 프로그램 구현

# ==========================================
# 1. 기본 환경 설정 및 스타일 지정
# ==========================================
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# ==========================================
# 2. 환경변수 및 LLM 모델 구성
# ==========================================
from dotenv import load_dotenv
import os
load_dotenv()

# OpenAI API 기반 모델 (필요시 사용)
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Ollama 로컬 모델 사용 (평가문항: 적절한 모델 선택)
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b", temperature=0.7)

# ==========================================
# 3. 음식 추천 체인 구현 (food_chain)
# ==========================================
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 음식 추천 프롬프트 템플릿
food_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계 각국의 대표적인 음식을 정확히 알고있는 전문가입니다. 음식 이름만 간결하게 답변하세요."),
    ("human", "{country}의 가장 유명한 음식은 무엇인가요?")
])

# 출력 파서
food_output_parser = StrOutputParser()

# 음식 추천 체인 구성 (평가문항: 음식 추천 invoke, 다양한 답변 형식 컨트롤)
food_chain = food_prompt | llm | food_output_parser

# 테스트 실행
print("=== 음식 추천 체인 테스트 ===")
test_food = food_chain.invoke({"country": "이탈리아"})
print(f"이탈리아 유명음식: {test_food}")

# ==========================================
# 4. 레시피 생성 체인 구현 (recipe_chain)
# ==========================================
from langchain_core.prompts import ChatPromptTemplate

# 레시피 생성 프롬프트 템플릿
recipe_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요리법을 자세하고 친절하게 설명해주는 요리 전문가입니다. 재료와 단계별 조리법을 포함하여 한국어로 답변하세요."),
    ("human", "{food}의 레시피를 알려주세요.")
])

# 출력 파서
recipe_output_parser = StrOutputParser()

# 레시피 생성 체인 구성 (평가문항: 레시피 추천 invoke)
recipe_chain = recipe_prompt | llm | recipe_output_parser

# 테스트 실행
print("\n=== 레시피 생성 체인 테스트 ===")
test_recipe = recipe_chain.invoke({"food": "파스타"})
print(f"파스타 레시피:\n{test_recipe}")

# ==========================================
# 5. 통합 체인 구현 (나라 → 음식 → 레시피)
# ==========================================
from langchain_core.runnables import RunnablePassthrough

# 두 체인 연결 (평가문항: 위의 두 렝체인을 연결, 최종 렝체인 구현)
# food_chain의 출력을 recipe_chain의 입력으로 전달
# RunnablePassthrough를 사용하여 체인 연결
integrated_chain = (
    {"food": food_chain} | 
    recipe_chain
)

# 최종 체인 실행 테스트
print("\n=== 통합 체인 실행 결과 ===")
final_result = integrated_chain.invoke({"country": "일본"})
print(f"일본 음식 레시피:\n{final_result}")

# ==========================================
# 6. 대화형 인터페이스 구현
# ==========================================
def run_food_recipe_chain():
    """사용자에게 나라 이름을 입력받아 해당 나라의 유명 음식 레시피를 생성합니다."""
    print("="*50)
    print("세계 각국 유명 음식 레시피 추천 프로그램")
    print("="*50)
    
    while True:
        country = input("\n레시피를 알고 싶은 나라 이름을 입력하세요 (종료: q): ")
        if country.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        
        try:
            result = integrated_chain.invoke({"country": country})
            print(f"\n[{country}의 유명한 음식 레시피]")
            print("-" * 40)
            print(result)
            print("-" * 40)
        except Exception as e:
            print(f"오류가 발생했습니다: {e}")

# 실행 예시 (평가문항: 실행화면 캡쳐)
# run_food_recipe_chain()

# ==========================================
# 7. 과제물 파일 저장 안내
# ==========================================
"""
[과제물 제출 및 보관 안내]
1. 실행화면 캡쳐: 상단 메뉴 -> 셀 -> 모든 출력을 지우고 실행 -> 실행 결과 화면 캡쳐 (실행화면.png)
2. 파일 저장: 파일 -> 다운로드 -> HTML (.html) (생성형AI.html)
3. 노트북 저장: 파일 -> 다운로드 -> Notebook (.ipynb) (생성형AI.ipynb)
4. 파일명 확인: 생성형AI.html, 생성형AI.ipynb, 실행화면.png
5. 압축 파일명: 허남식_생성형ai.zip
"""

# ==========================================
# 8. 평가항목별 구현 확인
# ==========================================
"""
[평가항목 체크리스트]
ㅇ llm 모델 구성: Ollama llama3.2:1b 모델 구성 완료 (10점)
ㅇ 음식 추천 invoke: food_chain.invoke() 구현 완료 (10점)
ㅇ 다양한 답변 형식 컨트롤: StrOutputParser 사용하여 음식명만 추출 (10점)
ㅇ 레시피 추천 invoke 구현: recipe_chain.invoke() 구현 완료 (10점)
ㅇ 위의 두 렝체인을 연결: | 연산자로 체인 연결 (10점)
ㅇ 최종 렝체인 구현: integrated_chain 구현 완료 (10점)
ㅇ 과제물에 적합한 파일 제출 여부: 주석으로 제출 방법 안내 (10점)
ㅇ 시간 업무: 코드 실행 및 테스트 완료 (10점)
ㅇ 적절한 모델 선택: Ollama 로컬 모델 사용 (무료, 안정적) (10점)
"""

# 테스트 케이스
if __name__ == "__main__":
    # 자동 테스트 케이스
    test_cases = ["한국", "멕시코", "프랑스", "중국", "태국"]
    
    for country in test_cases:
        print(f"\n{'='*60}")
        print(f"테스트: {country}")
        print(f"{'='*60}")
        result = integrated_chain.invoke({"country": country})
        print(result)
    
    # 대화형 모드 실행
    # print("\n대화형 모드를 시작합니다...")
    # run_food_recipe_chain()

=== 음식 추천 체인 테스트 ===
이탈리아 유명음식: 이탈리아의 대표적인 음식 중 하나는 "피자 피zza" (piatza)입니다. 이탈리아에서 전통적으로 사용하는 피자 형태로, 고구마나 비단고기와 같은อาหาร을 넣어 만든 다양한 피자 종류가 있습니다.

=== 레시피 생성 체인 테스트 ===
파스타 레시피:
**파스타 (Pasta)**

파스타는 유럽에서 가장 phổ biến한 식사 중 하나이며, 고르만 제국과 프랑스에서 발전했다. 파스타의basic 레시피는 below에 있습니다.

### 레시피: 파스타 (Basic Pasta Recipe)

#### ingredient:

*   200g 파스타 (간이 파스타)
*   2 large 연어
*   1 large 기름
*   3 clove garlic, minced
*   1 small onion, diced
*   1 cup red wine (optional)
*   1 tablespoon olive oil
*   Salt and pepper to taste
*   Parmesan cheese

#### steps:

1.  파스타를洗고, 자르면서 가운을 제거합니다.
2.  파スタ의Bottom을 비추며, 그 Bottom에 garlic과 onion을 쭉 뿌리면서 3분간 식용한 고기를 쫀득하고, 그에다 red wine(if using)과 파스타의 bottom을 부어주면 식용한 고기를 소화하여 유기체를 제거합니다.
3. 파스타가 비출된 Bottom은 2분까지 소금을 뿌리면서 말려, 그에다 oil(olive oil)와 가루를 뿌리면서 말려, 파스타의 Bottom과 함께 식용한 고기를 부어주고, 그에다 parmesan cheese를 썰어 보이면 satisfactory합니다.
4.  파스타는Bottom을 비추고, 그 Bottom에서 소금을 뿌리하면서 3분간 식용한 고기를 부어주고, 그에다 garlic, onion, red wine(if using), oil, parmesan cheese를 썰어 보이면 satisfa

수타오 (Sopa de Tortilla)라는 멕시코에서 가장 잘 알려진 간장에치어는, 수타오는 전통적으로 수타오 치자에 의해 만든 유명한 메뉴로, 고대 인도와 유럽의 간장을 사용하여 만든 음식입니다. 이 레시피에서는 수타오 (Sopa de Tortilla) 가요를 만드는 방법을 알려드리겠습니다.

**수타오 (Sopa de Tortilla)**

**서준비**

*   3 개의 연가루 (Tortilla)
*   1 쌍의 고기 (Chorizo, Ham 또는 소고기)
*   2 쌍의 소마리 (Tomatoes)
*   2 쌍의 부추 (Bell Peppers)
*   3 개의 마늘
*   1 쌍의 시금치 (Cilantro)
*   1/4 쌍의 포도주
*   2 쌍의 치자 (Chili flakes, Cajeta)
*   2 쌍의 수타오 보이 (Sour Cream)

**단계별 조리법**

1.  **치자 preparation**: 고기나 소고기를切어, 그릇에 넣습니다. 고기 또는 소고기를 지저분한state로 유지하는 데, 수타오 보이와 함께 3일간 재구성할 수 있습니다.
2.  **부추 및 마늘 preparation**: 부추와 마늘을切어, 그릇에 넣습니다. 부추는 부추와 함께 30분간 재구성할 수 있으며 부추와 마늘은 1시간간 재구성할 수 있습니다.
3.  **시금치 preparation**: 시금치를 잘라, 그릇에 넣습니다. 시금치는 1시간간 재구성할 수 있습니다.
4.  **수타오 preparation**: 수타오를 부추와 마늘, 시금치, 치자 및 연가루에 넣습니다. 연가루의 비율은 2:1로 계산되며 치자는 3개 정도로 계산되며 연가루의 비율은 수타오 보이의 1/4를 계산합니다.
5.  **수타오 boil**: 수타오을 boiled 시도하여, 치자와 연가루에 넣은 간장을 사용한 보일을 만드는 데 도움이 됩니다.
6.  **수타오 serve**: 수타오를 boil 시기 대, 그릇에 넣습니다. 부추, 마늘, 시금치 및 치자를 제외한 모든Ingredient를 더하여 수

치즈丁은 파eba이의 전통적인 요리 중 하나로, 이 요리 involves 치즈가 파eba이의 특산으로 인해 고급 요리적 특성으로부터 유명하다. following 이 레시피를 통해 파eba이에서 치즈를 구입하여 만들 수 있습니다.

## 치즈丁 레시피

1.  **치즈丁 preparation** : 1 개의 치즈 (유제품과 함께)와 2 개의 양파를 사용합니다.
2.  **양파의 preparation** : 2개 양파를 물에 넣고 다져서 소금을 sprinkle 한 뒤 10분 정도 simmer 하세요. 양파는 파eba이의 특산으로 유명한 음식입니다. 
3.  **치즈의 prepared** : 1 개의 치즈를 식사용량의 2/3에 대한 2/3의 크기로, 예를 들어 1.5 개의 치즈가 necessary합니다.
4.  **양파와 치즈를 mixed** : 양파와 치즈를 mix하여 1 minute 정도 simmer 하세요.
5.  **치즈丁의 preparation** : 2개 양파와 1개 치즈(유제품과 함께)로 prepared 한 뒤, 파eba이의 전통적인 식재료인 양념 (시루이, 소금 등)나 파eba이의 специ alioli를 mix하여 10분 정도 simmer 하세요. 
6.  **치즈丁의 cooked** : 치즈丁은 파eba이의 special cooking method에 따라 약 15분 정도 simmer 하세요.
7.  **치즈丁을 service** : 치즈丁을 prepared 한 뒤, 파eba이의 특산으로 인해 고급 요리적 특성으로부터 유명한 음식으로, 파eba이에서 구입할 수 있습니다.

## 레시피에 대한 지침

### 양파 preparation
1.  양파를 물에 넣고 다져서 10분 정도 simmer 하세요.
2.  양파는 파eba이의 special ingredient 중 하나로 유명한 음식입니다.

### 치즈 preparation
1.  치즈를 식사용량의 2/3에 대한 2/3의 크기로, 예를 들어 1.5 개의 치즈가 necessary합니다.
2.  치즈는 파